Pauli string commutation 

```
Reggio et al, Fast Partitioning of Pauli Strings into Commuting Families for Optimal Expectation Value Measurements of Dense Operators, 2023-06-07
```

In [22]:
from typing import *
from collections import OrderedDict
from itertools import combinations, combinations_with_replacement as re_combi, product
from functools import reduce
from sys import float_info
FLOAT_EPS = 1E4 * float_info.min
from pathlib import Path

import numpy as np
from scipy import linalg
import pandas as pd
from matplotlib import pyplot as plt
plt.rcParams.update({"font.serif": "Times New Roman"})


In [3]:
from pauli_op import PauliDecompose

In [4]:
PauliDecompose("XYYZ")

(2, 'IZZZ', 'XXXI')

In [20]:
# utils
def krons(oper_list): # Operator Kronecker delta
    return reduce(np.kron, oper_list)
def frobenius_inner(A, B): # Frobenius inner product.
    n, n2 = A.shape
    return np.trace((A.conj().T)@B)/(n)
def get_pauli_coefficient(A): # Get Pauli coefficients of the given Hermit matrix A.
    k, k2 = A.shape
    n = int(np.log2(k))
    p_fam, p_str = get_pauli_familiy(n)
    
    coef = {}
    for p_m, p_m_str in zip(p_fam, p_str):
        coef[p_m_str] = frobenius_inner(p_m, A)
    return coef

def get_coef(x_str, z_str): 
    # i coefficient in construction of general pauli-element from XZ elements.
    n = len(x_str)
    x_str = x_str.replace("X", "1")
    x_str = x_str.replace("I", "0")
    z_str = z_str.replace("Z", "1")
    z_str = z_str.replace("I", "0")
    
    x_int = int(x_str, 2)
    z_int = int(z_str, 2)
    
    y_pos = format(x_int&z_int, f"0{n}b")
    z_pos = format((x_int|z_int) - x_int, f"0{n}b")
    x_pos = format((x_int|z_int) - z_int, f"0{n}b")

    g_str = []
    for x,y,z in zip(x_pos, y_pos, z_pos):
        if x==y and y==z:
            g_str.append("I")
        elif x== "1":
            g_str.append("X")
        elif y == "1":
            g_str.append("Y")
        else:
            g_str.append("Z")
    return 1j**y_pos.count("1"), "".join(g_str)

In [21]:
I = np.eye(2)
pauli_X = np.array([[0, 1], [1, 0]])
pauli_Y = complex(0, 1)*np.array([[0, -1], [1, 0]])
pauli_Z = np.array([[1, 0], [0, -1]])

In [ ]:
# Sample
t_h = Hamiltonian()
commute_map = t_h.get_commute_map() # compatible graph
latin_matrix = t_h.get_latin_matrix() # get latin matrix

## Routines

1. Decompose the given hermite matrix as Pauli-polynomial.
   1. Generate n qubit pauli-matrices.
   2. Frobnius inner product.
   3. Kronecker product, tensor product on matrix form.
2. Compute Latin matrix
3. Generate compatible graph of Pauli-terms or local terms, adjacent matrix.
4. Saave and reload the profile

In [ ]:
float_tol = 1E-8
class Hamiltonian:
    def __init__(self, 
                 H:np.matrix, 
                 tols=(1E4*float_tol , float_tol), 
                 pauli_basis:Union[None, dict]=None):
        assert len(H.shape) ==2, f"H must be 2dim matrix. current: {H.shape}."
        n1, n2 = H.shape
        assert n1 == n2, f"Hamiltonian must be square matrix. Current:{(n1, n2)}."
        assert np.allclose(H, H.H, *tols), f"Hamiltonian must be a hermite matrix. Relative, absolute tolerance, {tols}."
        assert bin(8)[2:].count("1") == 1, f"Dimension must be a 2^n. Current:{n1}."
        
        self.Hamiltonian = H
        
        # None or Dataframe
        self.local_decomposition = self._check_decomposition(pauli_basis)  
        self.exist_decompositon = False if pauli_basis is None else True 
        self.x_family = None # save as integer 2dim matrix point the latin matrix.
        self.z_family = None
        self.coefficients = None # Latin matrix corresponding coefficient.
        
        self.qubit_num = len(bin(2)[3:]) # Consider a 1 bit position of 2^n integer.
    @staticmethod
    def get_total_hamiltonian(pstrs:dict):
        p_matrices = Hamiltonian.pstr_to_matrix(pstrs)
        pass
    @staticmethod
    def get_pauli_strings(qubit_num):
        n = int(qubit_num)
        assert n >0, "The given argument must be a positive natural number."
        
        p_xs =  get_pauli_family_matrix(n, fam="X")
        p_zx =  get_pauli_family_matrix(n, fam="Z")
        p_xs_str = get_pauli_family_string(n, fam="X")
        p_zs_str = get_pauli_family_string(n, fam="Z")

        p_g = []
        p_g_str =[]
        for x_i, x_str in zip(p_xs, p_xs_str):
            for z_j, z_str in zip(p_zs, p_zs_str):
                g = x_i@z_j

                g_coef, g_str = get_coef(x_str, z_str)

                p_g.append(g_coef*g)
                p_g_str.append(g_str)
        return p_g, p_g_str 
    #--------------------------------------------------------------
    def _check_decomposition(self, pauli_basis):
        if pauli_basis is None:
            return None
        pass    
    def get_decomposition(self, H:np.matrix, tol=float_tol ):
        pass
    def get_pauli_family_string(n, fam="Z"):
        return list(map(lambda x: "".join(x), product(f"I{fam}", repeat=int(n))))
    def get_pauli_family_matrix(n:int, fam="Z")->Iterable[np.matrix]:
        """Get pauli_family of `n` qubits of `fam` family. 

        Args:
            n (int): Number of qubits. The output matrices are :math:`2^n`.
            fam (str, optional): Type of Pauli-family of X, Y, or Z. Defaults to "Z".

        Returns:
            Iterable[np.matrix]: list of Pauli-matrices
        """
        G = pauli_Z if fam=="Z" else (pauli_X if fam=="X" else pauli_Y)

        return list(map(krons, product([I, G], repeat=int(n))))
        
    
    def decompose(self, tol=float_tol , replace = False):
        if self.exist_decompositon
            if not replace:
                return self.local_decomposition
        
        if replace:
            self.local_decomposition = self.get_decomposition(self.H, tol) 
            self.exist_decompositon = True
            return self.local_decomposition
        return self.get_decomposition(self.H, tol) 
    def save_as(self, filepath:Union[Path, str]):
        if isinstance(filepath, str):
            filepath = Path(filepath)
            
        pass
    #--------------------------------------------------------------
    @property
    def pauli_decomposition(self):
        if self.exist_decompositon:
            pass
        return None
    @property
    def xz_family(self):
        if self.exist_decompositon:
            pass
        return None
    @property
    def latin_matrix(self):
        if self.exist_decompositon:
            pass
        return None    
    #--------------------------------------------------------------
    @classmethod
    from from_latin_matrix(cls:Hamiltonian, 
                      l_matrix:np.matrix, 
                      coefficient:Union[np.matrix, None]=None):
        return cls()
    @classmethod
    from from_pauli_polynomial(cls:Hamiltonian, 
                               p_poly:Union[dict, np.ndarray], 
                               p_coef:Union[None, np.ndarray]=None):
        pass
    @classmethod
    from from_data(cls, file_path):
        pass
    #------------------------------
    

In [15]:
len(bin(2)[3:])

1